In [7]:
#%%capture cap --no-stderr
#uncomment above code (& last 2 lines) if u want output in a txt file
#This is an attempt at hard coding using numpy with verbose explanation
#pywraplp or other packages with inbuild linprog fns should be used if efficiency is the idea
import numpy as np
np.set_printoptions(precision=3, linewidth=10000)

def rdzero(M): #this is a function to round zeroes to prevent float inaccuracy problems
    for index, values in np.ndenumerate(M):
        if abs(values)<10**-4: #adjust this as necessary for precision
            index=list(index)
            M[index[0]][index[1]]=0

def checksolved(M,obj): #checks if Matrix is solved
    rdzero(M)
    for col in range(len(M[0])): #k is column no.
        stalling=0
        if M[0][col]>0 and col>obj-1:
            for x in range(len(M)-obj): #x is row no. excluding objective rows, this loop is to check if column is stalling
                if M[x+obj][-1]==0 and M[x+obj][col]>0:
                    stalling = 1
                    break
            if stalling==1:
                continue #skip to next column, note if all remaining columns are stalling then minima has reached
            else: #column is not stalling and can be minimized
                print('column '+'can still be optimized')
                print(M)
                return False
    print('optimized!')
    print(M)
    return True
#solves a matrix M in canonical form, where l is list of vars of bfs
#entries in l represent the column numbers which are the vars, all should be distinct
def solve_canon(M,cols,obj): #obj refers to number of objective function rows/columns 
    #checks if truly canonical form, mostly for troubleshooting
    for c in cols:
        check = 0
        for r in range(len(M)):
            if r == 0:
                if M[r][c]!=0:
                    print('entry in objective function row not 0')
                    print(M)
                    print(cols)
                    return 0
            elif M[r][c]!=0 and M[r][c]!=1:
                print('cell in var columns not equals 1 or 0')
                print(cols,r,c,M[r][c])
                print(M)
                return 0
            elif M[r][c]==1:
                check+=1
        if check!=1:
            print(str(check) + " of cells with 1, invalid canonical" )
            print(M)
            return 0
    print('Correct Canonical Form')
    #begin solving
    while checksolved(M,obj)==False:
        print('Not solved')
        print(M)
        for y in range(len(M[0])): #y is column number
            if M[0][y]>0 and y>obj-1:
                pivot_row = {}
                for x in range(len(M)-obj): #x is row no. excluding obj row
                    if M[x+obj][-1]==0 and M[x+obj][y]>0: 
                        #stalling column, we want to shift it only if the index is smaller (prevents cycling and will also help later when finding BFS)
                        for y2 in range(len(M[0])):
                            if M[x+obj][y2]==1 and M[0][y2]==0 and y2>y: #check if y2 is the column with var in bfs, and index of new one is smaller
                                M = M.astype('float64')
                                M[x+obj]=M[x+obj]*(1/M[x+obj][y]) #change pivot element to 1
                                rdzero(M)
                                for k in range(len(M)): #k is the row no. including obj row
                                    if k!=x+obj:
                                        M = M.astype('float64')
                                        M[k]+=M[x+obj]*(-M[k][y]/M[x+obj][y]) #stalling pivot completed
                                        rdzero(M)
                                        print('stalling pivot completed')
                                        print(M)
                                pivot_row={0:'stalled'}
                                break #no longer check the rest of the rows in current column as column is stalling
                    elif M[x+obj][y]>0:
                        pivot_row[x+obj]=M[x+obj][-1]/M[x+obj][y]
                if pivot_row=={0:'stalled'}: #checks if current column has stalled 
                    break #check if solved
                if pivot_row=={}: #all entries 0 or <0 which means no minima
                    print('NO MINIMUM')
                    print(M)
                    return 0
                index_min = min(pivot_row, key=lambda k: pivot_row[k]) #gives index of first min
                M = M.astype('float64')
                M[index_min]=M[index_min]*(1/M[index_min][y]) #scale row so pivot elem is now 1
                rdzero(M)
                for x in range(len(M)): #x is row no. including obj row
                    if x!=index_min:
                        M = M.astype('float64')
                        M[x]+=M[index_min]*(-M[x][y]/M[index_min][y]) #turning pivot column into 0s
                        rdzero(M)
                        print('pivot completed')
                        print(M)
                break #Pivot has completed and should check if solved
    print('Minima at '+str(M[0][-1]/M[0][0]))
    print(M)
    return M

def solve_general(M): #This function is if M is not in canonical form and need to search for BFS
    fakevars = len(M)-1 #number of fake variables to introduce
    print('Original matrix is:')
    print(M)
    
    for row in range(len(M)):#want to flip all negative to positive at last column
        if M[row][-1]<0:
            M[row]=-M[row]
    
    fakematrix = np.insert(np.identity(fakevars,dtype=np.float64),0,np.zeros(fakevars,dtype=np.float64),axis=0)
    fakematrix = np.transpose(fakematrix)
    M = np.insert(M,-1,fakematrix,axis=1) #inserting fake vars column
    M = np.insert(M,0,np.zeros(len(M)),axis=1) #insert new obj column
    
    newobj = [1] #new objective function row to be inserted on top of M
    for k in range(len(M[0])-2-fakevars): #0 for original vars
        newobj.append(0)
    for k in range(fakevars): #-1 for new fake vars
        newobj.append(-1) 
    newobj.append(0)
    newobj=np.array(newobj,dtype=np.float64)
    M=np.insert(M,0,newobj,axis=0) #inserting new objective function row at the top row
    
    print('New problem is: ')
    print(M)
    for x in range(len(M)): #x is row number, want to make it into canonical form
        if x>=2:
            M[0]+=M[x]
    print('Canonical form of new prob is: ')
    print(M)
    newcanonlist = [len(M[0])-k-2 for k in range(fakevars)]
    M=solve_canon(M,newcanonlist,2)
    if np.isscalar(M)==True: #if error occured in solve_canon
        return 0
    elif M[0][-1]!=0:
        print('No feasible solution')
        return 0
    else:
        print('BFS found')
    #No need to consider what to do if stalling prevented shifting from fakevars, due to algorithm above
    oldcanonlist=[] #canonical list for original/old problem
    for y in range(len(M[0])-fakevars-1): #y is column no. excluding fake vars and final column
        if y>=2:
            count=0
            for x in range(len(M)-2): #x is row no.
                if M[x+2][y]!=0 and M[x+2][y]!=1:
                    count=0
                    break
                if M[x+2][y]==1:
                    count+=1
            if count==1 and M[0][y]==0:
                oldcanonlist.append(y-1) #-1 because we are going to remove the fake objective function column later
    if len(oldcanonlist)==fakevars: #check if no issue
        M=M[1:]
        M=np.delete(M,slice(-fakevars-1,-1),axis=1)
        M=np.delete(M,0,axis=1)
        #obj function should not appear in restrictions!
        for x in range(len(M)):
            if x!=0 and M[x][0]!=0:
                print('Obj function appeared in restrictions!')
                print(M)
                return 0
        solve_canon(M,oldcanonlist,1)
    else:
        print('Error with varcount')
        print(fakevars)
        print(oldcanonlist)
        print(M)
        return 0

test1 = np.array([[1,-2,-3,-5,-2,-3,0,0,0],[0,1,1,2,1,3,-1,0,4],[0,2,-2,3,1,1,0,-1,3]])
test2 = np.array([[1,1,1,0,0,0,0],[0,2,1,1,0,0,4],[0,1,2,0,1,0,4],[0,1,-1,0,0,1,1]])
canon2 = [3,4,5]
test3 = np.array([[1,1,3,0,0,0],[0,1,-2,1,0,4],[0,-1,1,0,1,3]])
test4 = np.array([[-1,1,-1,0,0,0,0,0,0,0,0,0,0],[0,-1,1,3/25,7/25,11/25,3/5,-1,0,0,0,0,0],[0,-1,1,0,-1/25,2/25,1/5,0,-1,0,0,0,0],[0,-1,1,1/25,0,-7/25,-1/5,0,0,-1,0,0,0],[0,-1,1,-2/25,7/25,0,-3/5,0,0,0,-1,0,0],[0,-1,1,-1/5,1/5,3/5,0,0,0,0,0,-1,0],[0,0,0,1,1,1,1,0,0,0,0,0,1]])
test5 = np.array([[1,2,3,4,0],[0,3,2,1,10],[0,2,5,3,15]])
solve_canon(test2,canon2,1)
solve_general(test4)

#uncomment below code if u want it in a txt file
#with open('linprog_output.txt', 'w') as f:
#   f.write(cap.stdout)

Correct Canonical Form
column can still be optimized
[[ 1  1  1  0  0  0  0]
 [ 0  2  1  1  0  0  4]
 [ 0  1  2  0  1  0  4]
 [ 0  1 -1  0  0  1  1]]
Not solved
[[ 1  1  1  0  0  0  0]
 [ 0  2  1  1  0  0  4]
 [ 0  1  2  0  1  0  4]
 [ 0  1 -1  0  0  1  1]]
pivot completed
[[ 1.  0.  2.  0.  0. -1. -1.]
 [ 0.  2.  1.  1.  0.  0.  4.]
 [ 0.  1.  2.  0.  1.  0.  4.]
 [ 0.  1. -1.  0.  0.  1.  1.]]
pivot completed
[[ 1.  0.  2.  0.  0. -1. -1.]
 [ 0.  0.  3.  1.  0. -2.  2.]
 [ 0.  1.  2.  0.  1.  0.  4.]
 [ 0.  1. -1.  0.  0.  1.  1.]]
pivot completed
[[ 1.  0.  2.  0.  0. -1. -1.]
 [ 0.  0.  3.  1.  0. -2.  2.]
 [ 0.  0.  3.  0.  1. -1.  3.]
 [ 0.  1. -1.  0.  0.  1.  1.]]
column can still be optimized
[[ 1.  0.  2.  0.  0. -1. -1.]
 [ 0.  0.  3.  1.  0. -2.  2.]
 [ 0.  0.  3.  0.  1. -1.  3.]
 [ 0.  1. -1.  0.  0.  1.  1.]]
Not solved
[[ 1.  0.  2.  0.  0. -1. -1.]
 [ 0.  0.  3.  1.  0. -2.  2.]
 [ 0.  0.  3.  0.  1. -1.  3.]
 [ 0.  1. -1.  0.  0.  1.  1.]]
pivot completed
[[ 1.     0.

In [60]:
#This is an attempt at hard coding simplex algorithm without packages 

def ic(M,i,j): #interchange row functions, might not use?
    M[i], M[j]=M[j], M[i] 
    return M

def sm(M,s,r): #Multiply row r of Matrix M by scalar s
    for k in range(len(M[r])):
        M[r][k]=s*M[r][k]
    return M

def add(M,r1,r2,s): #Add scalar of row 2 to row 1
    for k in range(len(M[r1])):
        M[r1][k]+=s*M[r2][k]
    return M

def printmat(M): #prints Matrix in readable form
    string=''
    for row in range(len(M)):
        for cell in range(len(M[row])):
            string+=str("{:.2f}".format(M[row][cell]))+'\t'
        string+='\n'
    print(string)

#Inputs for minimizing a linear program (with all vars >= 0):
#0th row should be the objective function to minimize
#E.g. minimize X=2a+3b-4c should be rewritten as X-2a-3b+4c=0 and hence
#1, -2, -3, 4, ... ,0 in the 0th row
#subsequent linear restriction also of the same form with slack vars
#E.g. 2a + 3b - 4c <= 10 should include a slack var s to get
#2a + 3b - 4c + s = 10
#... , 2, 3, -4, 1, ... ,10


def checksolved(M,obj): #checks if Matrix is solved
    for k in range(len(M[0])): #k is column no.
        stalling=0
        if M[0][k]>0 and k>obj-1:
            for x in range(len(M)-obj): #x is row no. excluding objective rows, this loop is to check if column is stalling
                if M[x+obj][-1]==0 and M[x+obj][k]>0:
                    stalling = 1
                    print('stalling at ',x+obj,k, M[0][k])
                    break
            if stalling==1:
                continue #skip to next column, note if all remaining columns are stalling then minima has reached
            else: #column is not stalling and can be minimized
                printmat(M)
                return False
    return True

#solves a matrix M in canonical form, where l is list of vars of bfs
#entries in l represent the column numbers which are the vars, all should be distinct
def solve_canon(M,cols,obj): #obj refers to number of objective function rows/columns 
    #checks if truly canonical form, mostly for troubleshooting
    for k in cols:
        check = 0
        for r in range(len(M)):
            if r == 0:
                if M[r][k]!=0:
                    print('entry in objective function row not 0')
                    printmat(M)
                    print(cols)
                    return 0
            elif M[r][k]!=0 and M[r][k]!=1:
                print('cell in var columns not equals 1 or 0')
                print(cols,r,k,M[r][k])
                printmat(M)
                return 0
            elif M[r][k]==1:
                check+=1
        if check!=1:
            print(str(check) + " of cells with 1, invalid canonical" )
            printmat(M)
            return 0
    print('Correct Canonical Form')
    #begin solving
    while checksolved(M,obj)==False:
        printmat(M)
        print('Not solved')
        for y in range(len(M[0])): #y is column number
            if M[0][y]>0 and y>obj-1:
                pivot_row = {}
                for x in range(len(M)-obj): #x is row no. excluding obj row
                    if M[x+obj][-1]==0 and M[x+obj][y]>0: 
                        #stalling column, we want to shift it only if the index is smaller (prevents cycling and will also help later when finding BFS)
                        for y2 in range(len(M[0])):
                            if M[x+obj][y2]==1 and M[0][y2]==0 and y2>y: #check if y2 is the column with var in bfs, and index of new one is smaller
                                sm(M,1/M[x+obj][y],x+obj)
                                for k in range(len(M)): #k is the row no. including obj row
                                    if k!=x+obj:
                                        add(M,k,x+obj,-M[k][y]/M[x+obj][y]) #stalling pivot completed
                                pivot_row={0:'stalled'}
                                break #no longer check the rest of the rows in current column as column is stalling
                    elif M[x+obj][y]>0:
                        pivot_row[x+obj]=M[x+obj][-1]/M[x+obj][y]
                if pivot_row=={0:'stalled'}: #checks if current column has stalled 
                    break #check if solved
                if pivot_row=={}: #all entries 0 or <0 which means no minima
                    print('NO MINIMUM')
                    return 0
                index_min = min(pivot_row, key=lambda k: pivot_row[k]) #gives index of first min
                sm(M,1/M[index_min][y],index_min) #scale row so pivot elem is now 1
                for x in range(len(M)): #x is row no. including obj row
                    if x!=index_min:
                        add(M,x,index_min,-M[x][y]/M[index_min][y]) #turning pivot column into 0s
                break #Pivot has completed and should check if solved
    print('Minima at '+str(M[0][-1]/M[0][0]))
    printmat(M)
    return M
                   
def solve_general(M): #This function is if M is not in canonical form and need to search for BFS
    fakevars = len(M)-1 #number of fake variables to introduce
    print('Original matrix is:')
    printmat(M)
    newobj = [1] #new objective function row to be inserted on top of M
    for k in range(len(M[0])-1): #0 for original vars
        newobj.append(0)
    for k in range(fakevars): #-1 for new fake vars
        newobj.append(-1) 
    newobj.append(0)
    M.insert(0,newobj) #inserting new objective function row at the top row
    for x in range(len(M)):#x is row number, want to append fake vars to old matrix
        if x==1: #row of the original obj fn
            for y in range(fakevars): #add fake vars
                M[x].insert(-1,0) #inserts 0 into 2nd last position
            M[x].insert(0,0) # insert 0 at the beginning for new obj fn column
        if x>=2: #equations
            for y in range(fakevars):
                if x-2==y: 
                    if M[x][-1]<0: #if last cell of row is negative, flip signs
                        sm(M,-1,x)
                    M[x].insert(-1,1)
                else:
                    M[x].insert(-1,0)
            M[x].insert(0,0) # insert 0 at the beginning for new obj fn column
    print('New problem is: ')
    printmat(M)
    for x in range(len(M)): #x is row number, want to make it into canonical form
        if x>=2:
            add(M,0,x,1)
    print('Canonical form of new prob is: ')
    printmat(M)
    newcanonlist = [len(M[0])-k-2 for k in range(fakevars)]
    M=solve_canon(M,newcanonlist,2)
    if M==0: #if error occured in solve_canon
        return 0
    elif M[0][-1]!=0:
        print('No feasible solution')
        return 0
    else:
        print('BFS found')
    #No need to consider what to do if stalling prevented shifting from fakevars, due to algorithm above
    oldcanonlist=[] #canonical list for original/old problem
    for y in range(len(M[0])-fakevars-1): #y is column no. excluding fake vars and final column
        if y>=2:
            count=0
            for x in range(len(M)-2): #x is row no.
                if M[x+2][y]!=0:
                    count+=1
            if count==1 and M[0][y]==0:
                oldcanonlist.append(y-1) #-1 because we are going to remove the fake objective function column later
    if len(oldcanonlist)==fakevars: #check if no issue
        M=M[1:]
        for row in M:
            del row[-fakevars-1:-1] #delete fake vars
            del row[0]
        #obj function may appear in restrictions, hence we want to remove it
        for x in range(len(M)):
            if x!=0 and M[x][0]!=0:
                add(M,x,0,-M[x][0]/M[0][0])
        solve_canon(M,oldcanonlist,1)
    else:
        print('Error with varcount')
        print(fakevars)
        print(oldcanonlist)
        printmat(M)
        return 0
    
        
    

    
test1 = [[1,-2,-3,-5,-2,-3,0,0,0],[0,1,1,2,1,3,-1,0,4],[0,2,-2,3,1,1,0,-1,3]]
test2 = [[1,1,1,0,0,0,0],[0,2,1,1,0,0,4],[0,1,2,0,1,0,4],[0,1,-1,0,0,1,1]]
test3 = [[1,1,3,0,0,0],[0,1,-2,1,0,4],[0,-1,1,0,1,3]]
test4 = [[-1,1,-1,0,0,0,0,0,0,0,0,0,0],[0,-1,1,3/25,7/25,11/25,3/5,-1,0,0,0,0,0],[0,-1,1,0,-1/25,2/25,1/5,0,-1,0,0,0,0],[0,-1,1,1/25,0,-7/25,-1/5,0,0,-1,0,0,0],[0,-1,1,-2/25,7/25,0,-3/5,0,0,0,-1,0,0],[0,-1,1,-1/5,1/5,3/5,0,0,0,0,0,-1,0],[0,0,0,1,1,1,1,0,0,0,0,0,1]]
test5 = [[1,2,3,4,0],[0,3,2,1,10],[0,2,5,3,15]]
solve_general(test5)

#there are issues with rounding in this algorithm, resulting in 10^-17 numbers in supposedly 0 fields
#running this code will demonstrate the issue

Original matrix is:
1.00	2.00	3.00	4.00	0.00	
0.00	3.00	2.00	1.00	10.00	
0.00	2.00	5.00	3.00	15.00	

New problem is: 
1.00	0.00	0.00	0.00	0.00	-1.00	-1.00	0.00	
0.00	1.00	2.00	3.00	4.00	0.00	0.00	0.00	
0.00	0.00	3.00	2.00	1.00	1.00	0.00	10.00	
0.00	0.00	2.00	5.00	3.00	0.00	1.00	15.00	

Canonical form of new prob is: 
1.00	0.00	5.00	7.00	4.00	0.00	0.00	25.00	
0.00	1.00	2.00	3.00	4.00	0.00	0.00	0.00	
0.00	0.00	3.00	2.00	1.00	1.00	0.00	10.00	
0.00	0.00	2.00	5.00	3.00	0.00	1.00	15.00	

Correct Canonical Form
1.00	0.00	5.00	7.00	4.00	0.00	0.00	25.00	
0.00	1.00	2.00	3.00	4.00	0.00	0.00	0.00	
0.00	0.00	3.00	2.00	1.00	1.00	0.00	10.00	
0.00	0.00	2.00	5.00	3.00	0.00	1.00	15.00	

1.00	0.00	5.00	7.00	4.00	0.00	0.00	25.00	
0.00	1.00	2.00	3.00	4.00	0.00	0.00	0.00	
0.00	0.00	3.00	2.00	1.00	1.00	0.00	10.00	
0.00	0.00	2.00	5.00	3.00	0.00	1.00	15.00	

Not solved
1.00	0.00	0.00	3.67	2.33	-1.67	0.00	8.33	
0.00	1.00	0.00	1.67	3.33	-0.67	0.00	-6.67	
0.00	0.00	1.00	0.67	0.33	0.33	0.00	3.33	
0.00	0.00	0.00	3.

0